In [1]:
import qiskit
import qiskit_aer
import qiskit_nature
import qiskit_nature_pyscf
import qiskit_algorithms
import qiskit_nature_pyscf
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_aer import  AerSimulator
# from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from scipy.linalg import eigh
from qiskit_nature.second_q.circuit.library import HartreeFock
from scipy.linalg import expm
from qiskit.quantum_info import Statevector
from numpy import pi
import pylatexenc
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print(qiskit.__version__)
print(qiskit_aer.__version__)
print(qiskit_algorithms.__version__)
print(qiskit_nature.__version__)
print(qiskit_nature_pyscf.__version__)

1.4.0
0.17.0
0.3.1
0.7.2
0.4.0


In [3]:
# Define the Li molecule
molecule = MoleculeInfo(
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.0, 0.0, 1.59]),  # Approximate bond distance for He2
    multiplicity=1,  # Singlet state
    charge=0
)

# Set up the PySCF driver
mdriver = PySCFDriver.from_molecule(
    molecule=molecule,
    basis="sto3g"
)


# You can now use the driver to run a calculation
electronic_structure_problem = mdriver.run()

In [4]:
mapper = JordanWignerMapper()
qubit_hamiltonian = mapper.map(electronic_structure_problem.second_q_ops()[0])

In [7]:
from KQSCI_module import *

In [8]:
evolved_state_file = "state_probabilities.txt"
sample_space_size = 3
kqsci_size = 3
num_qubits = 12
state_probabilities = read_states(evolved_state_file)
eigenvalues, eigenvectors = kqsci_eign(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)

In [9]:
0.998447567774 + eigenvalues

array([-7.88023582, -7.12145814, -6.79896824])

In [10]:
evolved_state_file = "state_probabilities.txt"
state_probabilities = read_states(evolved_state_file)
max_d = 3
max_k = 4
with open("LiH_KQSCI_result.txt", "a") as file:
    for j in range(3, max_k):
        for i in range(1, max_d):
            
            sample_space_size = i
            kqsci_size = j
            num_qubits = 12
            
            eigenvalues, eigenvectors = kqsci_eign(state_probabilities, num_qubits, sample_space_size, kqsci_size, qubit_hamiltonian)
            gs_energy = eigenvalues[0] + 0.998447567774
            error = (7.882472287557 + eigenvalues[0] + 0.998447567774) / 7.882472287557 * 100
            file.write(f"sample_space_size(d): {sample_space_size}, kqsci_size(k): {kqsci_size}, gs_energy: {gs_energy}, error: {error}\n")
            file.flush()  # Ensure the data is written to the file immediately

In [11]:
def selection_full(state_probabilities):
    result = []
    print("Before normalising the states:")
    for time in sorted(state_probabilities.keys()):
        # Append all the states from all times
        for state, probability in state_probabilities[time].items():
            if state not in result:
                result.append(state)
            else:
                continue
    return result

In [12]:
states_chis = selection_full(state_probabilities)
len(states_chis)

Before normalising the states:


59

In [13]:
states= states_chis[:20]

In [14]:
eignva, eignve = kqsci_solver_full(qubit_hamiltonian, states, "E_sub_matrix.txt")

In [15]:
eignva[0] + 0.998447567774

np.float64(-7.88242611309071)

In [16]:
from KQSCI_module import kqsci_solver_full_parallel

In [19]:
states = states_chis[:30]
states

['000011000011',
 '100001100001',
 '100001000101',
 '000101100001',
 '001001001001',
 '010001010010',
 '010001010001',
 '000110000110',
 '001010001010',
 '000101000101',
 '100001000011',
 '100010100010',
 '000101000011',
 '000110000011',
 '010010010001',
 '001001001010',
 '000011000101',
 '010010010010',
 '000011100001',
 '000011000110',
 '100010000011',
 '001010001001',
 '100100000011',
 '000011100100',
 '000110100010',
 '100001100100',
 '100100100001',
 '011000011000',
 '101000101000',
 '100010100001']

In [20]:
eigenvalues, eigenvectors = kqsci_solver_full_parallel(qubit_hamiltonian, states, "E_sub_matrix_parallel.txt", -1)

In [21]:
eigenvalues[0] + 0.998447567774

np.float64(-7.882452899871752)

In [22]:
states = states_chis[:40]

In [23]:
eigenvalues, eigenvectors = kqsci_solver_full_parallel(qubit_hamiltonian, states, "E_sub_matrix_parallel_40.txt", -1)

In [25]:
eigenvalues[0] + 0.998447567774

np.float64(-7.88246001689081)

In [27]:
states = states_chis[:10]
eigenvalues, eigenvectors = kqsci_solver_full_parallel(qubit_hamiltonian, states, "E_sub_matrix_parallel_10.txt", -1)
eigenvalues[0] + 0.998447567774

np.float64(-7.881936929500559)